In [1]:
!pip install keras==2.2.5
!pip install tensorflow==1.14.0

     |████████████████████████████████| 336 kB 8.5 MB/s 
     |████████████████████████████████| 50 kB 4.7 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.6.0
    Uninstalling keras-2.6.0:
      Successfully uninstalled keras-2.6.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.6.0 requires keras~=2.6, but you have keras 2.2.5 which is incompatible.
     |████████████████████████████████| 109.3 MB 52 kB/s 
     |████████████████████████████████| 488 kB 69.5 MB/s 
     |████████████████████████████████| 3.1 MB 49.9 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.6.0
    Uninstalling tensorflow-estimator-2.6.0:
      Successfully uninstalled tensorflow-estimator-2.6.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.6.0
    Unins

In [2]:
import tensorflow
import keras
print(tensorflow.__version__)
print("keras version:", keras.__version__)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0
keras version: 2.2.5


/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [3]:
import re
import matplotlib.pyplot as plt
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


train.txt

![image](https://user-images.githubusercontent.com/44194558/137693162-d51fb4c2-416b-4325-a33b-21daff3e0f05.png)


### 데이터 불러오기 

In [5]:
sentences = []
buff_sentence = [] # 단어, 개체명 태깅 정보

with open('/content/drive/My Drive/train.txt', 'r') as f_r:
    # 문장 불러오기
    data = f_r.readlines()  
    # 각 문장에 대한 처리
    for sentence in data:
        if len(sentence) == 0 or sentence.startswith('-DOCSTART-') or sentence[0] == "\n":
            if len(buff_sentence) > 0:
                sentences.append(buff_sentence)
                buff_sentence = []
            continue

        splits = sentence.split(' ')
        splits[-1] = re.sub(r'\n', '', splits[-1]) # 줄바꿈 표시 \n 제거
        word = splits[0].lower() # 단어들은 소문자로 바꿔서 저장
        buff_sentence.append([word, splits[-1]]) 

In [6]:
print(sentences[0])

[['eu', 'B-ORG'], ['rejects', 'O'], ['german', 'B-MISC'], ['call', 'O'], ['to', 'O'], ['boycott', 'O'], ['british', 'B-MISC'], ['lamb', 'O'], ['.', 'O']]


In [7]:
sentences_info = [] # 단어 변수
tags_info = [] # 태그 변수

In [8]:
for sent in sentences:
    word, tag = zip(*sent) # * : unlist
    sentences_info.append(list(word))
    tags_info.append(list(tag))

In [9]:
print(*sentences[0])

['eu', 'B-ORG'] ['rejects', 'O'] ['german', 'B-MISC'] ['call', 'O'] ['to', 'O'] ['boycott', 'O'] ['british', 'B-MISC'] ['lamb', 'O'] ['.', 'O']


In [10]:
print("첫 번째 문장:", sentences_info[0])
print("첫 번째 태그 정보:", tags_info[0])

첫 번째 문장: ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
첫 번째 태그 정보: ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']


In [11]:
print("열 한 번째 문장:", sentences_info[10])
print("열 한 번째 태그 정보:",tags_info[10])

열 한 번째 문장: ['spanish', 'farm', 'minister', 'loyola', 'de', 'palacio', 'had', 'earlier', 'accused', 'fischler', 'at', 'an', 'eu', 'farm', 'ministers', "'", 'meeting', 'of', 'causing', 'unjustified', 'alarm', 'through', '"', 'dangerous', 'generalisation', '.', '"']
열 한 번째 태그 정보: ['B-MISC', 'O', 'O', 'B-PER', 'I-PER', 'I-PER', 'O', 'O', 'O', 'B-PER', 'O', 'O', 'B-ORG', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


### 전처리 : Tokenize & Sequence 변환


In [12]:
vocab_size = 5000  # 5000개의 단어만 사용
src_tokenizer = Tokenizer(num_words=vocab_size, oov_token='OOV')  # 문장 데이터
src_tokenizer.fit_on_texts(sentences_info)

tar_tokenizer = Tokenizer() # 태그 데이터
tar_tokenizer.fit_on_texts(tags_info)

In [13]:
# 시퀀스 변환 (단어 인덱스 집합)
x_train = src_tokenizer.texts_to_sequences(sentences_info)
y_train = tar_tokenizer.texts_to_sequences(tags_info)

In [17]:
print('원본 문장 :', sentences_info[0])
print('시퀀스로 변환된 문장 :', x_train[0])

print('원본 태그 :', tags_info[0])
print('시퀀스로 변환된 태그 :', y_train[0])

원본 문장 : ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
시퀀스로 변환된 문장 : [989, 1, 205, 629, 7, 3939, 216, 1, 3]
원본 태그 : ['B-ORG', 'O', 'B-MISC', 'O', 'O', 'O', 'B-MISC', 'O', 'O']
시퀀스로 변환된 태그 : [4, 1, 7, 1, 1, 1, 7, 1, 1]


In [18]:
index2word = src_tokenizer.index_word  # 단어 인덱스를 저장하는 딕셔너리
index2tag = tar_tokenizer.index_word

decoded = []

for index in x_train[0]:
    decoded.append(index2word[index])

print("기존 문장: {}".format(sentences_info[0]))
print("Vocabulary에 없어 OOV 처리된 단어: {}".format(decoded))

기존 문장: ['eu', 'rejects', 'german', 'call', 'to', 'boycott', 'british', 'lamb', '.']
Vocabulary에 없어 OOV 처리된 단어: ['eu', 'OOV', 'german', 'call', 'to', 'boycott', 'british', 'OOV', '.']


In [27]:
# 고정된 길이의 시퀀스로 변환
max_len = 80
x_train_padded = pad_sequences(x_train, padding='post', maxlen=max_len)
y_train_padded = pad_sequences(y_train, padding='post', maxlen=max_len)

print(x_train_padded[0])
print(y_train_padded[0])

[ 989    1  205  629    7 3939  216    1    3    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0]
[4 1 7 1 1 1 7 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0]


In [35]:
x_train, x_test, y_train, y_test = train_test_split(x_train_padded, y_train_padded, test_size=.2,
random_state=555)

print(len(x_train))
print(len(y_train))

11232
11232


In [36]:
print(y_train[0].shape)

(80,)


In [38]:
tag_size = len(tar_tokenizer.word_index) + 1
y_train = to_categorical(y_train, num_classes=tag_size)
y_test = to_categorical(y_test, num_classes=tag_size)

In [39]:
print(y_train[0].shape)

(80, 10)


In [34]:
print('훈련 샘플 문장의 크기: {}'.format(x_train.shape))
print('훈련 샘플 레이블의 크기: {}'.format(y_train.shape))
print('테스트 샘플 문장의 크기: {}'.format(x_test.shape))
print('테스트 샘플 레이블의 크기: {}'.format(y_test.shape))

훈련 샘플 문장의 크기: (11232, 80)
훈련 샘플 레이블의 크기: (11232, 80, 10)
테스트 샘플 문장의 크기: (2809, 80)
테스트 샘플 레이블의 크기: (2809, 80, 10)


### 모델링

In [42]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Bidirectional, TimeDistributed
from keras.optimizers import Adam

model = Sequential()
# mask_zero=True : 패딩된 값을 마스킹하여 네트워크의 뒤로 전달되지 않도록 (인위적으로 패딩된 부분이 학습에 영향을 미치지 않게 하도록)
model.add(Embedding(input_dim=vocab_size, output_dim=128, input_length=max_len, mask_zero=True))  
model.add(Bidirectional(LSTM(256, return_sequences=True)))
model.add(TimeDistributed(Dense(tag_size, activation='softmax')))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 80, 128)           640000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 80, 512)           788480    
_________________________________________________________________
time_distributed_2 (TimeDist (None, 80, 10)            5130      
Total params: 1,433,610
Trainable params: 1,433,610
Non-trainable params: 0
_________________________________________________________________


embedding 계층의 출력은 (batch_size, max_len, output_dim = embedding_dim) = (None, 80, 128)

5000개 단어(vocab_size)를 128 차원의 임베딩 노드로 전환 : 128 x 5000 = 64000

788480 = (256 x (128 + 256) + 256) x 4 x 2 = (LSTM 노드 수 x (입력 임베딩 차원 + LSTM 노드 수) + bias) x gate 수 x 2 (양방향)

In [45]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.001), metrics=['accuracy'])

In [46]:
hist = model.fit(x_train, y_train, batch_size=128, epochs=1, validation_data=(x_test, y_test))


Train on 11232 samples, validate on 2809 samples
Epoch 1/1
11232/11232 [==============================] - 154s 14ms/step - loss: 0.8959 - acc: 0.8242 - val_loss: 0.5854 - val_acc: 0.8299


### 결과

In [47]:
print("train loss: ", hist.history['loss'])
print("train accuracy: ", hist.history['acc'])
print("val loss: ", hist.history['val_loss'])
print("val accuracy: ", hist.history['val_acc'])

train loss:  [0.8959357295280848]
train accuracy:  [0.8241946862803565]
val loss:  [0.5854463567297499]
val accuracy:  [0.8299064717075502]


In [48]:
print("\n 테스트 정확도: %.4f" % (model.evaluate(x_test, y_test)[1]))

2809/2809 [==============================] - 18s 6ms/step

 테스트 정확도: 0.8294
